In [1]:
import cogent3
from cogent3 import get_app
from phylim.apps import phylim
import libs
import paths

In [ ]:
folder_in = paths.DATA_APES114 + 'introns/'
in_dstore = cogent3.open_data_store(folder_in, suffix='fa', mode='r')

print(in_dstore.describe)

Directory datastore
record type      number
-----------------------
completed          1587
not_completed         0
logs                  0
-----------------------


In [ ]:
#perform a codon alignment and remove stop codons
loader = get_app("load_aligned", moltype="dna")
omit_degs = get_app("omit_degenerates", moltype="dna")
rename = libs.renamer_aligned()
concat = get_app("concat", moltype="dna")

app = loader + omit_degs + rename
#By using this renamer_aligned function I throw away sequences with paralogs
results_allpos = [r for r in app.as_completed(in_dstore[:], parallel=True) if r]
alpos_alns = concat(results_allpos)
alpos_alns.source = "introns_alignments"
alpos_alns

   0%|          |00:00<?

,0
Chimpanzee,TTTTTGTAGCTGGGATTACAGGCATGCACCACCATGCCTGGCTAATTTTTGTATTTTTAG
Human,............................................................
Gorilla,................................T...........................


In [4]:
#Get distances of concatenated sequences 
#time_het=max sets a 36 params substitution model
sm = get_app("model", "GN", time_het="max", optimise_motif_probs=True, show_progress=True)
result_intergenic = sm(alpos_alns)

checker = get_app("phylim")
checked = checker(result_intergenic)
lengthstree = result_intergenic.lf.get_lengths_as_ens()
print("Is model identifiable? " + str(checked.is_identifiable) + "\n")
print("Model results : \n" + str(result_intergenic.lf) + "\n")
print("Distances: " + str(lengthstree))

   0%|          |00:00<?

   0%|          |00:00<?

Is model identifiable? True

Model results : 
GN
log-likelihood = -11266720.1739
number of free parameters = 39
edge          parent    length     A>C     A>G     A>T     C>A     C>G
----------------------------------------------------------------------
Gorilla       root        0.01    0.94    4.25    0.73    1.25    1.42
Human         root        0.01    0.99    3.99    0.82    1.42    1.41
Chimpanzee    root        0.01    1.01    3.66    0.87    1.28    1.37
----------------------------------------------------------------------

continued: 
 C>T     G>A     G>C     G>T     T>A     T>C
--------------------------------------------
6.57    6.67    1.37    1.30    0.78    4.19
7.44    7.52    1.41    1.41    0.77    3.90
7.03    6.94    1.37    1.28    0.83    3.68
--------------------------------------------

   A       C       G       T
----------------------------
0.27    0.23    0.23    0.27
----------------------------

Distances: {'Gorilla': np.float64(0.012332691914139379), 'Hum